# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [21]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)
df = pd.get_dummies(df, columns=['home_ownership','verification_status','application_type','pymnt_plan','initial_list_status',
                               'hardship_flag','debt_settlement_flag'])

months_num = {
    "Jan-2019": 1,
    "Feb-2019": 2,
    "Mar-2019": 3,
    "Apr-2019": 4,
    "May-2019": 5,
    "Jun-2019": 6,
    "Jul-2019": 7,
    "Aug-2019": 8,
    "Sep-2019": 9,
    "Oct-2019": 10,
    "Nov-2019": 11,
    "Dec-2019": 12,
}

df['issue_d'] = df['issue_d'].apply(lambda x: months_num[x])
df['next_pymnt_d'] = df['next_pymnt_d'].apply(lambda x: months_num[x])

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(df["loan_status"])
df["loan_status_le"] = label_encoder.transform(df["loan_status"])
df.drop(columns='loan_status', axis=1, inplace=True)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,annual_inc,issue_d,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,verification_status_Source Verified,verification_status_Verified,application_type_Individual,application_type_Joint App,pymnt_plan_n,initial_list_status_f,initial_list_status_w,hardship_flag_N,debt_settlement_flag_N,loan_status_le
0,10500.0,0.1719,375.35,66000.0,3,27.24,0.0,0.0,8.0,0.0,...,1,0,1,0,1,0,1,1,1,1
1,25000.0,0.2000,929.09,105000.0,3,20.23,0.0,0.0,17.0,1.0,...,0,1,1,0,1,0,1,1,1,1
2,20000.0,0.2000,529.88,56000.0,3,24.26,0.0,0.0,8.0,0.0,...,0,1,1,0,1,0,1,1,1,1
3,10000.0,0.1640,353.55,92000.0,3,31.44,0.0,1.0,10.0,1.0,...,0,1,1,0,1,0,1,1,1,1
4,22000.0,0.1474,520.39,52000.0,3,18.76,0.0,1.0,14.0,0.0,...,0,0,1,0,1,0,1,1,1,1


## Split the Data into Training and Testing

In [22]:
# Create our features
X = df.drop(columns='loan_status_le', axis=1).copy()

# Create our target
y = df["loan_status_le"].values.reshape(-1, 1)

In [23]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,issue_d,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,application_type_Individual,application_type_Joint App,pymnt_plan_n,initial_list_status_f,initial_list_status_w,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,1.726172,21.778153,0.217766,0.497697,12.587340,0.126030,...,0.478007,0.373992,0.148001,0.860340,0.139660,1.0,0.123879,0.876121,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,0.743862,20.199244,0.718367,0.758122,6.022869,0.336797,...,0.499520,0.483865,0.355104,0.346637,0.346637,0.0,0.329446,0.329446,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,1.000000,0.000000,0.000000,0.000000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,1.000000,13.890000,0.000000,0.000000,8.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,1.0,0.000000,1.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,2.000000,19.760000,0.000000,0.000000,11.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,1.0,0.000000,1.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,2.000000,26.660000,0.000000,1.000000,16.000000,0.000000,...,1.000000,1.000000,0.000000,1.000000,0.000000,1.0,0.000000,1.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,3.000000,999.000000,18.000000,5.000000,72.000000,4.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.0


In [24]:
# Check the balance of our target values
# YOUR CODE HERE
y

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [25]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [26]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [27]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)

In [28]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [36]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=78)
rf_model = rf_model.fit(X_train_scaled, y_train)
predictions = rf_model.predict(X_test_scaled)

In [37]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
balanced_accuracy_score(y_test, predictions)

0.6663745764692137

In [38]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, predictions)

array([[   29,    58],
       [   10, 17108]])

In [39]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.74      0.33      1.00      0.46      0.58      0.31        87
          1       1.00      1.00      0.33      1.00      0.58      0.36     17118

avg / total       1.00      1.00      0.34      1.00      0.58      0.36     17205



In [40]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.08038416909803944, 'total_rec_int'),
 (0.07244445418418362, 'total_rec_prncp'),
 (0.07212424753342449, 'total_pymnt_inv'),
 (0.06441261126854798, 'total_pymnt'),
 (0.05526766386164843, 'last_pymnt_amnt'),
 (0.02106691064297415, 'installment'),
 (0.018998161427414388, 'out_prncp'),
 (0.018216639745348366, 'out_prncp_inv'),
 (0.01812739366241642, 'dti'),
 (0.017327147226459224, 'mo_sin_old_il_acct'),
 (0.016513626336478876, 'mo_sin_old_rev_tl_op'),
 (0.014820185055480765, 'loan_amnt'),
 (0.014720584555127692, 'max_bal_bc'),
 (0.014720000016676157, 'revol_bal'),
 (0.014566014737581557, 'tot_cur_bal'),
 (0.014018777308706171, 'bc_open_to_buy'),
 (0.013920213220429406, 'issue_d'),
 (0.013883306223607504, 'total_rec_late_fee'),
 (0.013865971607174796, 'bc_util'),
 (0.013527284878858509, 'annual_inc'),
 (0.013521253386503819, 'total_il_high_credit_limit'),
 (0.013239616504041518, 'total_rev_hi_lim'),
 (0.013061827291639683, 'total_bal_il'),
 (0.012933309893472102, 'int_rate'),
 (0.0126975

### Easy Ensemble Classifier

In [42]:
# Train the Classifier
# YOUR CODE HERE
from imblearn.ensemble import EasyEnsembleClassifier
easy_model = EasyEnsembleClassifier(n_estimators=100, random_state=76)
easy_model.fit(X_train_scaled, y_train)
easy_predictions = easy_model.predict(X_test_scaled)

In [43]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
balanced_accuracy_score(y_test, predictions)

0.6663745764692137

In [44]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, predictions)

array([[   29,    58],
       [   10, 17108]])

In [45]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.74      0.33      1.00      0.46      0.58      0.31        87
          1       1.00      1.00      0.33      1.00      0.58      0.36     17118

avg / total       1.00      1.00      0.34      1.00      0.58      0.36     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    Both had the same.

2. Which model had the best recall score?

    Both had the same

3. Which model had the best geometric mean score?

    Both had the same

4. What are the top three features?

    total_rec_int, total_rec_prncp, total_pymnt_inv